# Make testing way easier by using **tox**

### Also, a small introduction to testing automation, best practices and cats 🐈

### TODO: explain that this is the second presentation of the cycle
This is the second presentation of the testing cycle. The first one was about mocking.

![Tox logo](https://tox.wiki/en/rewrite/_static/tox.svg)
## What is tox?
#### *And (most importantly) how is it going to make my life any better?*

In [ ]:
Why should I start using tox?

It has been adopted already by many major projects:
- [flask](https://github.com/pallets/flask/blob/master/tox.ini)    
- [numpy](https://github.com/numpy/numpy/blob/master/tox.ini)    
- [requests](https://github.com/psf/requests/blob/master/tox.ini)    
- [scipy](https://github.com/scipy/scipy/blob/master/tox.ini)

It is supported (and encouraged) by [Azure Pipelines](https://docs.microsoft.com/en-us/azure/devops/pipelines/ecosystems/python?view=azure-devops#run-tests-with-tox).

```
- job:

  pool:
    vmImage: 'ubuntu-latest'
  strategy:
    matrix:
      Python27:
        python.version: '2.7'
      Python37:
        python.version: '3.5'
      Python38:
        python.version: '3.8'

  steps:
  - task: UsePythonVersion@0
    displayName: 'Use Python $(python.version)'
    inputs:
      versionSpec: '$(python.version)'

  - script: pip install tox
    displayName: 'Install Tox'

  - script: tox -e py
    displayName: 'Run Tox'
```

### Useful links:
[Catfacts (API reference)](https://catfact.ninja/)    
[Previous presentation (Notebook)](https://github.com/jean-n92/triangle-community-presentations/blob/minor/refresh-and-refactor/notebooks/presentation-pytest.ipynb)    
[This application (Git repo)](https://github.com/jean-n92/triangle-community-presentations)    
[tox (Docs)](https://tox.wiki/en/rewrite/index.html)
[Automating Build, Test and Release Workflows with tox (Video)](https://youtu.be/PrAyvH-tm8E)